In [1]:
import pandas as pd
from PIL import Image
import cv2
import sys
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
# # load image
# img_path = 'Lettuce_Roots_2022/2022_03_16/2309.jpg'

# plot = img_path.split('/')[-1][-9:-4]
# date = img_path.split('/')[-2]
# print(date)
# img = cv2.imread(img_path)

# # Cropping an image
# cropped_image = img[200:1450, 700:2200] #[pixels in the vertical direction, pixels in the horizontal direction]

# # Set out directory and create if not already in working directory
# out_dir = os.path.join('Cropped_Images', str(plot))

# if not os.path.isdir(out_dir):
#     os.makedirs(out_dir)

# # Save the cropped image
# cv2.imwrite(f"{out_dir}/{plot}_Cropped.jpg", cropped_image)

In [3]:
def crop_img(img_path):
    plot = img_path.split('/')[-1][-9:-4]

    img = cv2.imread(img_path)

    # Cropping an image
    cropped_image = img[200:1450, 700:2200] #[pixels in the vertical direction, pixels in the horizontal direction]

    # Set out directory and create if not already in working directory
    out_dir = os.path.join('Cropped_Images', str(plot))

    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)

    # Save the cropped image
    cv2.imwrite(f"{out_dir}/{plot}_Cropped.jpg", cropped_image)
    print('Image Cropped and Saved')


## Removing Background from Cropped Images

In [54]:
# img_path = 'Cropped_Images/2022_03_16/0311_Cropped.jpg'

# plot = img_path.split('/')[-1][-9:-4]

# date = img_path.split('/')[-2]

# img = cv2.imread(img_path)


In [33]:
# # Convert Image into HSV space
# img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

# # extract hue channel from the image
# hue = img_hsv[:,:,0]

# # plotting hue channel
# plt.title('hue channel')
# plt.imshow(hue)
# plt.show()

# # calculate a histogram with 180 bins, one for each color
# hist, _ = np.histogram(hue, bins=180, normed=True)
# plt.title("hue histogram")
# plt.plot(hist)
# plt.show()

In [53]:
# lower = np.array([0, 52, 72], dtype = "uint16") ## Pink Tag Values are approximately (217, 149, 172; R, G, B)
# upper = np.array([100, 255, 255], dtype = "uint16")
# mask = cv2.inRange(img, lower, upper)

# Mask = cv2.bitwise_not(mask) ## Inverting the mask

# cv2.imwrite('mask0.png', Mask)

In [120]:
def create_mask(img_path, img):
    # Establish Plot Name
    plot = img_path.split('/')[-1].split('.')[-2]

    date = img_path.split('/')[-2]
    
    # # Load Image
    # img = cv2.imread(img_path)

    # Set Boundaries for mask
    lower = np.array([0, 52, 72], dtype = "uint16") ## Pink Tag Values are approximately (217, 149, 172; R, G, B)
    upper = np.array([95, 255, 255], dtype = "uint16")
    mask = cv2.inRange(img, lower, upper)

    ## Inverting the mask
    Mask = cv2.bitwise_not(mask)

    # Set out directory and create if not already in working directory
    out_dir = os.path.join('Mask_Images', str(date))

    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)

    # Save the mask
    cv2.imwrite(f'{out_dir}/{plot}_mask.png', Mask)
    return Mask

In [121]:
def remove_background(img_path, img, Mask):
    # Establish Plot Name
    plot = img_path.split('/')[-1].split('.')[-2]
    # Establish Date
    date = img_path.split('/')[-2]

    _, Mask = cv2.threshold(Mask, thresh=180, maxval=255, type=cv2.THRESH_BINARY)

    # copy where we'll assign the new values
    no_back = np.copy(img)

    # boolean indexing and assignment based on mask
    no_back[(Mask==255).all(-1)] = [0,0,0]

    # Set out directory and create if not already in working directory
    out_dir = os.path.join('Clean_Images', str(date))

    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)

    # Save the mask
    cv2.imwrite(f'{out_dir}/{plot}_clean.png', no_back)

    return no_back

In [106]:
# mask = cv2.imread('mask0.png')

# _, mask = cv2.threshold(mask, thresh=180, maxval=255, type=cv2.THRESH_BINARY)
# # copy where we'll assign the new values
# no_back = np.copy(img)
# # boolean indexing and assignment based on mask
# no_back[(mask==255).all(-1)] = [0,0,0]

# fig, ax = plt.subplots(1,2,figsize=(12,6))
# ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# ax[1].imshow(cv2.cvtColor(no_back, cv2.COLOR_BGR2RGB))

In [127]:
## This will be the main

img_path = 'Cropped_Images/2022_03_16/0311_Cropped.jpg'

plot = img_path.split('/')[-1].split('.')[-2]

date = img_path.split('/')[-2]

img = cv2.imread(img_path)

create_mask(img_path, img)

Mask = cv2.imread(f'Mask_Images/{date}/{plot}_mask.png')

no_back = remove_background(img_path, img, Mask)